In [1]:
!pip install pandas statsmodels
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
!pip install arch
from statsmodels.tsa.stattools import adfuller, kpss
from arch.unitroot import PhillipsPerron

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 종속변수

In [2]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/insurance.csv').dropna()
df.columns.values[0] = 'date'
df_col = ['date', '변액종신', '변액연금', '변액유니버셜']
df = df[df_col]

df.head()

,date,변액종신,변액연금,변액유니버셜
1,2018_11,0.00850,0.01002,0.00987
2,2018_12,0.00846,0.01009,0.00993
3,2019_01,0.01570,0.01061,0.01151
4,2019_02,0.01583,0.00976,0.01062
5,2019_03,0.01229,0.01074,0.01272


## 기본통계량

In [3]:
df.describe()

,변액종신,변액연금,변액유니버셜
count,50.000000,50.000000,50.000000
mean,0.008498,0.010531,0.008625
std,0.004817,0.002464,0.002467
min,0.004200,0.006490,0.004050
25%,0.005565,0.008700,0.006428
50%,0.006835,0.010125,0.009205
75%,0.008490,0.011845,0.010597
max,0.026990,0.019270,0.012720


## 차분

In [4]:
df_col = ['변액종신', '변액연금', '변액유니버셜']

for col in df_col:
    df[f'{col}_1차로그차분'] = np.log(df[col]).diff()
    df[f'{col}_2차로그차분'] = df[f'{col}_1차로그차분'].diff()
    df[f'{col}_2차로그차분_스케일'] = df[f'{col}_2차로그차분'] * 100

df.head(10)

,date,변액종신,변액연금,변액유니버셜,변액종신_1차로그차분,변액종신_2차로그차분,변액종신_2차로그차분_스케일,변액연금_1차로그차분,변액연금_2차로그차분,변액연금_2차로그차분_스케일,변액유니버셜_1차로그차분,변액유니버셜_2차로그차분,변액유니버셜_2차로그차분_스케일
1,2018_11,0.00850,0.01002,0.00987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018_12,0.00846,0.01009,0.00993,-0.004717,NaN,NaN,0.006962,NaN,NaN,0.006061,NaN,NaN
3,2019_01,0.01570,0.01061,0.01151,0.618312,0.623029,62.302853,0.050252,0.043290,4.329038,0.147656,0.141595,14.159512
4,2019_02,0.01583,0.00976,0.01062,0.008246,-0.610065,-61.006538,-0.083505,-0.133757,-13.375667,-0.080477,-0.228133,-22.813295
5,2019_03,0.01229,0.01074,0.01272,-0.253121,-0.261367,-26.136711,0.095683,0.179187,17.918724,0.180437,0.260914,26.091375
6,2019_04,0.01565,0.01176,0.01170,0.241685,0.494806,49.480594,0.090729,-0.004954,-0.495384,-0.083587,-0.264023,-26.402326
7,2019_05,0.01579,0.00997,0.01104,0.008906,-0.232779,-23.277908,-0.165123,-0.255852,-25.585221,-0.058064,0.025523,2.552292
8,2019_06,0.01463,0.00956,0.01036,-0.076303,-0.085209,-8.520852,-0.041993,0.123131,12.313050,-0.063573,-0.005509,-0.550900
9,2019_07,0.01611,0.01201,0.01173,0.096366,0.172669,17.266860,0.228152,0.270145,27.014477,0.124197,0.187770,18.777023
10,2019_08,0.01493,0.01016,0.01104,-0.076068,-0.172434,-17.243357,-0.167281,-0.395433,-39.543310,-0.060625,-0.184822,-18.482205


## 단위근 검정 : ADF, PP, KSPP

In [5]:
def adf_test(data):
    adf_result = adfuller(data.dropna())
    print(f'ADF Statistic: {adf_result[0]}')
    print(f'p-value: {adf_result[1]}')
    print('Critical Values:')
    for key, value in adf_result[4].items():
        print(f'   {key}: {value}')

def pp_test(data):
    pp_result = PhillipsPerron(data.dropna())
    print('\n PP Test Results:')
    print(f'   Test Statistic: {pp_result.stat}')
    print(f'   p-value: {pp_result.pvalue}')
    print(f'   Lags Used: {pp_result.lags}')

def kpss_test(data, regression='c'):
    kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)
    print('\n KPSS Test Results:')
    print(f'   KPSS Statistic: {kpss_stat}')
    print(f'   p-value: {p_value}')
    print(f'   Lags Used: {lags}')
    print('   Critical Values:')
    for key, value in critical_values.items():
        print(f'      {key}: {value}')

In [6]:
#변액종신
data = df['변액종신_2차로그차분_스케일']

adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.779229558253111
p-value: 5.172482362553349e-07
Critical Values:
   1%: -3.596635636000432
   5%: -2.933297331821618
   10%: -2.6049909750566895

 PP Test Results:
   Test Statistic: -35.26938634337961
   p-value: 0.0
   Lags Used: 10

 KPSS Test Results:
   KPSS Statistic: 0.5000000000000085
   p-value: 0.04166666666666474
   Lags Used: 47
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [7]:
#변액연금
data = df['변액연금_2차로그차분_스케일']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.789189336530779
p-value: 4.91765859292833e-07
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338

 PP Test Results:
   Test Statistic: -23.405256360615773
   p-value: 0.0
   Lags Used: 10

 KPSS Test Results:
   KPSS Statistic: 0.14902638575791177
   p-value: 0.1
   Lags Used: 14
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [8]:
#변액유니버셜
data = df['변액유니버셜_2차로그차분_스케일']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -6.5619572818533705
p-value: 8.338296495734582e-09
Critical Values:
   1%: -3.596635636000432
   5%: -2.933297331821618
   10%: -2.6049909750566895

 PP Test Results:
   Test Statistic: -41.97547162513315
   p-value: 0.0
   Lags Used: 10

 KPSS Test Results:
   KPSS Statistic: 0.27758620098775666
   p-value: 0.1
   Lags Used: 27
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [9]:
df_filtered = df[~df['date'].isin(['2018/10', '2018/11', '2018/12'])]
df_filtered[['date', '변액종신_2차로그차분_스케일', '변액연금_2차로그차분_스케일', '변액유니버셜_2차로그차분_스케일']].to_csv('종속변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('종속변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 독립변수

In [10]:
df = pd.read_csv('/content/drive/MyDrive/EWHA/SEED/2024-2/SEED_24_2_논문2조/독립변수.csv').drop(columns=['Unnamed: 6'])

In [11]:
# 공백이나 특수문자 제거
df.columns = df.columns.str.strip()  # 양쪽 공백 제거
df.columns = df.columns.str.replace('(', '')  # 괄호 제거
df.columns = df.columns.str.replace(')', '')  # 괄호 제거

# 다시 열 이름 확인
print(df.columns)

Index(['date', 'CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_증가'], dtype='object')


In [12]:
df.describe()

,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가
count,51.000000,51.000000,51.000000,51.000000,51.000000
mean,1.502745,3.568627,99.447059,102.245216,2450.248431
std,0.826692,0.518263,1.291875,3.393543,424.456606
min,0.630000,2.600000,96.400000,98.884000,1754.640000
25%,0.780000,3.150000,98.800000,99.641000,2102.595000
50%,1.470000,3.600000,99.600000,100.330000,2293.610000
75%,1.820000,3.900000,100.250000,103.955000,2798.330000
max,4.020000,4.800000,101.400000,109.260000,3296.680000


In [13]:
df_col = ['CD91일', '실업률', '동행지수순환변동치', '총지수', 'KOSPI_증가']

for col in df_col:
    df[f'{col}_1차로그차분'] = np.log(df[col]).diff()
    df[f'{col}_2차로그차분'] = df[f'{col}_1차로그차분'].diff()

df.head(10)

,date,CD91일,실업률,동행지수순환변동치,총지수,KOSPI_증가,CD91일_1차로그차분,CD91일_2차로그차분,실업률_1차로그차분,실업률_2차로그차분,동행지수순환변동치_1차로그차분,동행지수순환변동치_2차로그차분,총지수_1차로그차분,총지수_2차로그차분,KOSPI_증가_1차로그차분,KOSPI_증가_2차로그차분
0,2018/10,1.68,4.0,100.1,100.041,2029.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018/11,1.71,3.8,99.9,99.330,2096.86,0.017700,NaN,-0.051293,NaN,-0.002000,NaN,-0.007132,NaN,0.032558,NaN
2,2018/12,1.91,3.6,99.8,98.988,2041.04,0.110610,0.092910,-0.054067,-0.002774,-0.001002,0.000998,-0.003449,0.003683,-0.026982,-0.059539
3,2019/01,1.87,3.9,99.9,98.884,2204.85,-0.021165,-0.131775,0.080043,0.134110,0.001002,0.002003,-0.001051,0.002398,0.077200,0.104182
4,2019/02,1.87,3.9,99.4,99.311,2195.44,0.000000,0.021165,0.000000,-0.080043,-0.005018,-0.006019,0.004309,0.005360,-0.004277,-0.081477
5,2019/03,1.90,3.8,99.3,99.121,2140.67,0.015915,0.015915,-0.025975,-0.025975,-0.001007,0.004011,-0.001915,-0.006224,-0.025264,-0.020987
6,2019/04,1.86,4.1,99.1,99.481,2203.59,-0.021277,-0.037193,0.075986,0.101961,-0.002016,-0.001010,0.003625,0.005540,0.028969,0.054233
7,2019/05,1.84,3.9,99.4,99.652,2041.74,-0.010811,0.010466,-0.050010,-0.125996,0.003023,0.005039,0.001717,-0.001908,-0.076285,-0.105254
8,2019/06,1.80,3.9,99.4,99.491,2130.62,-0.021979,-0.011168,0.000000,0.050010,0.000000,-0.003023,-0.001617,-0.003334,0.042611,0.118896
9,2019/07,1.68,3.9,99.5,99.187,2024.55,-0.068993,-0.047014,0.000000,0.000000,0.001006,0.001006,-0.003060,-0.001443,-0.051066,-0.093676


In [14]:
data = df['CD91일_2차로그차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -11.314352537809295
p-value: 1.2169738493434652e-20
Critical Values:
   1%: -3.5745892596209488
   5%: -2.9239543084490744
   10%: -2.6000391840277777

 PP Test Results:
   Test Statistic: -14.690475580508135
   p-value: 3.055133163320823e-27
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.10830920493479528
   p-value: 0.1
   Lags Used: 7
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [15]:
data = df['실업률_2차로그차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.863244107667398
p-value: 3.3717300970806777e-07
Critical Values:
   1%: -3.5885733964124715
   5%: -2.929885661157025
   10%: -2.6031845661157025

 PP Test Results:
   Test Statistic: -29.209474513529486
   p-value: 0.0
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.17089921146481227
   p-value: 0.1
   Lags Used: 15
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [16]:
data = df['동행지수순환변동치_2차로그차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -4.602234991232484
p-value: 0.0001279499418503528
Critical Values:
   1%: -3.596635636000432
   5%: -2.933297331821618
   10%: -2.6049909750566895

 PP Test Results:
   Test Statistic: -8.923231232939106
   p-value: 1.0275896926282184e-14
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.0439779960475966
   p-value: 0.1
   Lags Used: 0
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)


In [17]:
data = df['총지수_2차로그차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.423996919338537
p-value: 3.0154913231716327e-06
Critical Values:
   1%: -3.584828853223594
   5%: -2.9282991495198907
   10%: -2.6023438271604937

 PP Test Results:
   Test Statistic: -14.556035181850241
   p-value: 4.879525082558951e-27
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.44653653293831813
   p-value: 0.05709632200934564
   Lags Used: 40
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [18]:
data = df['KOSPI_증가_2차로그차분']
adf_test(data)
pp_test(data)
kpss_test(data)

ADF Statistic: -5.303535749643879
p-value: 5.384062754439779e-06
Critical Values:
   1%: -3.5925042342183704
   5%: -2.931549768951162
   10%: -2.60406594375338

 PP Test Results:
   Test Statistic: -23.496432047508186

<ipython-input-5-06c07ab290e4>:17: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  kpss_stat, p_value, lags, critical_values = kpss(data.dropna(), regression=regression)



   p-value: 0.0
   Lags Used: 11

 KPSS Test Results:
   KPSS Statistic: 0.1929909789699695
   p-value: 0.1
   Lags Used: 15
   Critical Values:
      10%: 0.347
      5%: 0.463
      2.5%: 0.574
      1%: 0.739


In [19]:
df_filtered = df[~df['date'].isin(['2018/10', '2018/11', '2018/12'])]

df_filtered[['date', 'CD91일_2차로그차분', '실업률_2차로그차분', '동행지수순환변동치_2차로그차분', '총지수_2차로그차분', 'KOSPI_증가_2차로그차분']].to_csv('독립변수.csv', index=False, encoding='utf-8-sig')

from google.colab import files
files.download('독립변수.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>